# 要約 
このJupyter Notebookは、「LMSYS - Chatbot Arena」コンペティションにおける人間の好み予測に関する問題を解決するためのものです。具体的には、大規模言語モデル（LLM）を用いて、ユーザーからのプロンプトに対して選択された応答の好ましさを予測します。

### 主な内容とアプローチ
1. **ライブラリのインストール**: 
   - `bitsandbytes`, `transformers`, `tokenizers`, `peft` などのライブラリを使用して、LLMやトークナイザー、パラメータの効率的なトレーニングをサポートします。
   - また、分析のために`numpy`や`pandas`も利用されます。

2. **データの前処理**:
   - データセットはCSVファイルから読み込まれ、NaN値や重複データの削除が行われます。
   - 各応答の長さを計算し、これを元にデータをソートします。

3. **モデルの設定とトレーニング**:
   - LLaMAモデルに基づいて系列分類用のモデルを設定し、LoRA（Low-Rank Adaptation）を用いてパラメータを効率的にトレーニングします。
   - 設定構成（`Config`クラス）に基づいてトレーニング戦略が定義され、複数の分割に対する検証（交差検証）が行われます。

4. **トークナイザーのカスタマイズ**:
   - プロンプトと応答を組み合わせたテキストをトークナイズするためのカスタムトークナイザーを使用しています。

5. **メトリクスの計算**:
   - 予測精度と対数損失（log loss）が評価され、これらの結果を用いてモデルの性能を測定します。

6. **トレーニングと評価の反復**:
   - 訓練バッチごとにトレーニングを実行し、各エポック後に評価を行う構成になっています。

このノートブックでは、特にLLaMAモデルとLoRAを組み合わせて効率的な学習を実現し、Kaggleの競技における大規模言語モデルのトレーニングと評価に関する具体的なアプローチが示されています。

---


# 用語概説 
以下は、提供されたJupyter Notebookで使用されている専門用語の簡単な解説です。特に、初心者にとって馴染みのない用語や、実務での経験がないと理解しにくいものに焦点を当てています。

1. **bitsandbytes**: メモリ使用を最適化するためのライブラリで、特に大規模なモデルを効率的に訓練する際に小さいビット数でパラメータを表現することが可能です。

2. **peft (Parameter Efficient Fine-Tuning)**: ファインチューニングを行う際に、モデルの一部分（例えばアダプタ層）だけを調整する手法です。全体のモデルを訓練するのではなく、パラメータの数を削減しつつ性能を維持することを目指します。

3. **LoRA (Low-Rank Adaptation)**: 大規模モデルを適応させる際に、低ランクの行列分解を利用してネットワークの特定の部分だけを調整するアプローチです。これにより、少ないパラメータで効率的に適応が可能になります。

4. **gradient accumulation**: バッチサイズを物理的に増やすことなく、複数のバッチの勾配を累積する手法です。これにより、GPUのメモリ制約を越えて大規模なバッチトレーニングが可能になります。

5. **softmax**: 数値の集合を確率として解釈できるように変換するための関数です。出力ベクトルに対して使用すると、各要素のエクスポネンシャル関数を計算し、その合計で各要素を割ることで、合計が1になる確率ベクトルを得られます。

6. **EvalPrediction**: 評価時の予測値と真のラベルを格納するためのオブジェクトです。評価指標の計算などに利用されます。

7. **packing**: 入力データのトークン数を最大限に活用するためにデータをグループ化し、パディングを最小限に抑える処理です。効率的なバッチ処理を実現するために使用される技術です。

8. **SFT (Supervised Fine-Tuning)**: 監視付きのファインチューニング手法で、ラベル付きデータを用いてモデルの性能を向上させるために行います。特に、特定のタスクに対するモデルの適応を意味します。

9. **torch_dtype**: PyTorchにおいて、モデルで使用するデータ型を指定するオプションです。例えば、`torch.float16`は半精度浮動小数点数を意味し、メモリ使用量を削減するために使用されます。

10. **TaskType**: 特定のタスクに関連するモジュールを特定する列挙型で、ここでは系列分類 (`SEQ_CLS`) のようなタスクタイプを指定します。

11. **pretraining_tp (Tensor Parallelism)**: 大規模なモデルを効率よく訓練するために、モデルの計算を複数のGPUに分散させる手法です。これにより、メモリの制約を緩和し、もより大きなモデルを訓練可能にします。

12. **dropout**: ニューラルネットワークの訓練中に特定のニューロンをランダムに無効にすることで、過学習を防ぐ技法です。

13. **warmup steps**: 学習率を徐々に増加させるための初期ステップで、通常、訓練の初期段階でモデルが安定するように助けます。

これらの用語は、特に特定の技術やフレームワークでのみ使用されているため、初心者が理解するのが難しい場合があります。学習を進める上で、これらの用語の理解が助けになるでしょう。

---


<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install -q -U bitsandbytes --no-index --find-links ../input/llama3-1-dependencies/dependencies/
!pip install -q -U transformers --no-index --find-links ../input/llama3-1-dependencies/dependencies/
!pip install -q -U tokenizers --no-index --find-links ../input/llama3-1-dependencies/dependencies/
!pip install -q -U peft --no-index --find-links ../input/llama3-1-dependencies/dependencies/
```

</div>
<div class="column-right">

# 日本語訳

```python
!pip install -q -U bitsandbytes --no-index --find-links ../input/llama3-1-dependencies/dependencies/
!pip install -q -U transformers --no-index --find-links ../input/llama3-1-dependencies/dependencies/
!pip install -q -U tokenizers --no-index --find-links ../input/llama3-1-dependencies/dependencies/
!pip install -q -U peft --no-index --find-links ../input/llama3-1-dependencies/dependencies/
```

</div>
</details>

In [ ]:
!pip install -q -U bitsandbytes --no-index --find-links ../input/llama3-1-dependencies/dependencies/
!pip install -q -U transformers --no-index --find-links ../input/llama3-1-dependencies/dependencies/
!pip install -q -U tokenizers --no-index --find-links ../input/llama3-1-dependencies/dependencies/
!pip install -q -U peft --no-index --find-links ../input/llama3-1-dependencies/dependencies/

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install -U trl
```

</div>
<div class="column-right">

# 日本語訳

```python
!pip install -U trl
```

</div>
</details>

In [ ]:
!pip install -U trl

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
```

</div>
<div class="column-right">

# 日本語訳

```python
# このPython 3環境には多くの便利な分析ライブラリがインストールされています
# これはkaggle/python Dockerイメージによって定義されています: https://github.com/kaggle/docker-python
# 例えば、ここにいくつかの便利なパッケージをロードする例があります

import numpy as np # 線形代数ライブラリ
import pandas as pd # データ処理、CSVファイルのI/O（例: pd.read_csv）

# 入力データファイルは読み取り専用の"../input/"ディレクトリにあります
# 例えば、これを実行すると（クリックするかShift+Enterを押す）、入力ディレクトリ内のすべてのファイルがリストされます

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# 現在のディレクトリ(/kaggle/working/)に最大20GBまで書き込むことができ、"Save & Run All"を使ってバージョンを作成すると出力として保存されます
# 一時ファイルは/kaggle/temp/に書き込むこともできますが、そのファイルは現在のセッションの外では保存されません
```

</div>
</details>

In [ ]:
# このPython 3環境には多くの便利な分析ライブラリがインストールされています
# これはkaggle/python Dockerイメージによって定義されています: https://github.com/kaggle/docker-python
# 例えば、ここにいくつかの便利なパッケージをロードする例があります

import numpy as np # 線形代数ライブラリ
import pandas as pd # データ処理、CSVファイルのI/O（例: pd.read_csv）

# 入力データファイルは読み取り専用の"../input/"ディレクトリにあります
# 例えば、これを実行すると（クリックするかShift+Enterを押す）、入力ディレクトリ内のすべてのファイルがリストされます

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# 現在のディレクトリ(/kaggle/working/)に最大20GBまで書き込むことができ、"Save & Run All"を使ってバージョンを作成すると出力として保存されます
# 一時ファイルは/kaggle/temp/に書き込むこともできますが、そのファイルは現在のセッションの外では保存されません

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import os
import copy
from dataclasses import dataclass

import numpy as np
import torch
from datasets import Dataset
from transformers import (
    DataCollatorWithPadding,
    LlamaForSequenceClassification,
    LlamaTokenizerFast,
    PreTrainedTokenizerBase,
    EvalPrediction,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType, PeftModel
from sklearn.metrics import log_loss, accuracy_score
```

</div>
<div class="column-right">

# 日本語訳

```python
import os
import copy
from dataclasses import dataclass

import numpy as np
import torch
from datasets import Dataset
from transformers import (
    DataCollatorWithPadding,
    LlamaForSequenceClassification,
    LlamaTokenizerFast,
    PreTrainedTokenizerBase,
    EvalPrediction,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType, PeftModel
from sklearn.metrics import log_loss, accuracy_score
```

</div>
</details>

In [ ]:
import os
import copy
from dataclasses import dataclass

import numpy as np
import torch
from datasets import Dataset
from transformers import (
    DataCollatorWithPadding,
    LlamaForSequenceClassification,
    LlamaTokenizerFast,
    PreTrainedTokenizerBase,
    EvalPrediction,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType, PeftModel
from sklearn.metrics import log_loss, accuracy_score

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
@dataclass
class Config:
    output_dir: str = "output"
    checkpoint: str = "/kaggle/input/unsloth-meta-llama-3.1-8b-bnb-4bit/transformers/default/1/Meta-Llama-3.1-8B-bnb-4bit"
    max_length: int = 2048
    n_splits: int = 5
    fold_idx: int = 0
    optim_type: str = "adamw_8bit"
    per_device_train_batch_size: int = 4
    gradient_accumulation_steps: int = 4
    per_device_eval_batch_size: int = 8
    n_epochs: int = 1
    freeze_layers: int = 16  # there're 32 layers in total, we don't add adapters to the first 16 layers
    lr: float = 2e-4
    warmup_steps: int = 20
    lora_r: int = 4
    lora_alpha: float = lora_r * 2
    lora_dropout: float = 0.05
    lora_bias: str = "none"

config = Config()
```

</div>
<div class="column-right">

# 日本語訳

```python
@dataclass
class Config:
    output_dir: str = "output"  # 出力ディレクトリの設定
    checkpoint: str = "/kaggle/input/unsloth-meta-llama-3.1-8b-bnb-4bit/transformers/default/1/Meta-Llama-3.1-8B-bnb-4bit"  # チェックポイントのパス
    max_length: int = 2048  # 最大長さの設定
    n_splits: int = 5  # データの分割数
    fold_idx: int = 0  # 現在の折り目インデックス
    optim_type: str = "adamw_8bit"  # 最適化手法
    per_device_train_batch_size: int = 4  # デバイスごとのトレーニングバッチサイズ
    gradient_accumulation_steps: int = 4  # 勾配蓄積ステップ
    per_device_eval_batch_size: int = 8  # デバイスごとの評価バッチサイズ
    n_epochs: int = 1  # エポック数
    freeze_layers: int = 16  # レイヤーを凍結する数（全体で32レイヤーがあるため最初の16レイヤーにはアダプターを追加しない）
    lr: float = 2e-4  # 学習率
    warmup_steps: int = 20  # ウォームアップステップ
    lora_r: int = 4  # LoRAの設定
    lora_alpha: float = lora_r * 2  # LoRAのアルファ
    lora_dropout: float = 0.05  # LoRAのドロップアウト率
    lora_bias: str = "none"  # LoRAのバイアス設定

config = Config()  # 設定を生成
```

</div>
</details>

In [ ]:
@dataclass
class Config:
    output_dir: str = "output"  # 出力ディレクトリの設定
    checkpoint: str = "/kaggle/input/unsloth-meta-llama-3.1-8b-bnb-4bit/transformers/default/1/Meta-Llama-3.1-8B-bnb-4bit"  # チェックポイントのパス
    max_length: int = 2048  # 最大長さの設定
    n_splits: int = 5  # データの分割数
    fold_idx: int = 0  # 現在の折り目インデックス
    optim_type: str = "adamw_8bit"  # 最適化手法
    per_device_train_batch_size: int = 4  # デバイスごとのトレーニングバッチサイズ
    gradient_accumulation_steps: int = 4  # 勾配蓄積ステップ
    per_device_eval_batch_size: int = 8  # デバイスごとの評価バッチサイズ
    n_epochs: int = 1  # エポック数
    freeze_layers: int = 16  # レイヤーを凍結する数（全体で32レイヤーがあるため最初の16レイヤーにはアダプターを追加しない）
    lr: float = 2e-4  # 学習率
    warmup_steps: int = 20  # ウォームアップステップ
    lora_r: int = 4  # LoRAの設定
    lora_alpha: float = lora_r * 2  # LoRAのアルファ
    lora_dropout: float = 0.05  # LoRAのドロップアウト率
    lora_bias: str = "none"  # LoRAのバイアス設定

config = Config()  # 設定を生成

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
lora_config = LoraConfig(
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    target_modules=["q_proj", "k_proj", "v_proj"],
    layers_to_transform=[i for i in range(32) if i >= config.freeze_layers],
    lora_dropout=config.lora_dropout,
    bias=config.lora_bias,
    task_type=TaskType.SEQ_CLS,
)
```

</div>
<div class="column-right">

# 日本語訳

```python
lora_config = LoraConfig(
    r=config.lora_r,  # LoRAのr値を設定
    lora_alpha=config.lora_alpha,  # LoRAのアルファを設定
    target_modules=["q_proj", "k_proj", "v_proj"],  # 変換対象のモジュール
    layers_to_transform=[i for i in range(32) if i >= config.freeze_layers],  # 変換するレイヤーを選択
    lora_dropout=config.lora_dropout,  # LoRAのドロップアウト率
    bias=config.lora_bias,  # LoRAのバイアス設定
    task_type=TaskType.SEQ_CLS,  # タスクタイプを定義（系列分類）
)
```

</div>
</details>

In [ ]:
lora_config = LoraConfig(
    r=config.lora_r,  # LoRAのr値を設定
    lora_alpha=config.lora_alpha,  # LoRAのアルファを設定
    target_modules=["q_proj", "k_proj", "v_proj"],  # 変換対象のモジュール
    layers_to_transform=[i for i in range(32) if i >= config.freeze_layers],  # 変換するレイヤーを選択
    lora_dropout=config.lora_dropout,  # LoRAのドロップアウト率
    bias=config.lora_bias,  # LoRAのバイアス設定
    task_type=TaskType.SEQ_CLS,  # タスクタイプを定義（系列分類）
)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
tokenizer = LlamaTokenizerFast.from_pretrained(config.checkpoint)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True  # We'll add <eos> at the end
tokenizer.padding_side = "right"
```

</div>
<div class="column-right">

# 日本語訳

```python
tokenizer = LlamaTokenizerFast.from_pretrained(config.checkpoint)  # トークナイザーをチェックポイントから初期化
tokenizer.pad_token_id = tokenizer.eos_token_id  # パディントークンIDを終端トークンIDに設定
tokenizer.pad_token = tokenizer.eos_token  # パディントークンを終端トークンに設定
tokenizer.add_eos_token = True  # 文の終わりに<eos>トークンを追加
tokenizer.padding_side = "right"  # パディングを右側に設定
```

</div>
</details>

In [ ]:
tokenizer = LlamaTokenizerFast.from_pretrained(config.checkpoint)  # トークナイザーをチェックポイントから初期化
tokenizer.pad_token_id = tokenizer.eos_token_id  # パディントークンIDを終端トークンIDに設定
tokenizer.pad_token = tokenizer.eos_token  # パディントークンを終端トークンに設定
tokenizer.add_eos_token = True  # 文の終わりに<eos>トークンを追加
tokenizer.padding_side = "right"  # パディングを右側に設定

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
model = LlamaForSequenceClassification.from_pretrained(
    config.checkpoint,
    num_labels=3,
    torch_dtype=torch.float16,
    device_map="auto"
)
```

</div>
<div class="column-right">

# 日本語訳

```python
model = LlamaForSequenceClassification.from_pretrained(
    config.checkpoint,
    num_labels=3,  # クラスの数を指定
    torch_dtype=torch.float16,  # 使用するテンソルのデータ型を指定
    device_map="auto"  # デバイスマップの自動設定
)
```

</div>
</details>

In [ ]:
model = LlamaForSequenceClassification.from_pretrained(
    config.checkpoint,
    num_labels=3,  # クラスの数を指定
    torch_dtype=torch.float16,  # 使用するテンソルのデータ型を指定
    device_map="auto"  # デバイスマップの自動設定
)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model
```

</div>
<div class="column-right">

# 日本語訳

```python
model.config.use_cache = False  # キャッシュの使用を無効に設定
model = prepare_model_for_kbit_training(model)  # モデルをkbitトレーニング用に準備
model = get_peft_model(model, lora_config)  # PEFTモデルを取得
model
```

</div>
</details>

In [ ]:
model.config.use_cache = False  # キャッシュの使用を無効に設定
model = prepare_model_for_kbit_training(model)  # モデルをkbitトレーニング用に準備
model = get_peft_model(model, lora_config)  # PEFTモデルを取得
model

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
model.print_trainable_parameters()
```

</div>
<div class="column-right">

# 日本語訳

```python
model.print_trainable_parameters()  # 訓練可能なパラメータを出力
```

</div>
</details>

In [ ]:
model.print_trainable_parameters()  # 訓練可能なパラメータを出力

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False
model.config.pretraining_tp = 1
```

</div>
<div class="column-right">

# 日本語訳

```python
model.config.pad_token_id = tokenizer.pad_token_id  # パディントークンIDを設定
model.config.use_cache = False  # キャッシュの使用を無効に設定
model.config.pretraining_tp = 1  # 前処理のためのテンソル並列化設定
```

</div>
</details>

In [ ]:
model.config.pad_token_id = tokenizer.pad_token_id  # パディントークンIDを設定
model.config.use_cache = False  # キャッシュの使用を無効に設定
model.config.pretraining_tp = 1  # 前処理のためのテンソル並列化設定

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import pandas as pd
df = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
df = df.dropna()
df = df.drop_duplicates(subset=['response_a', 'response_b'], keep=False)
df["len"] = df["prompt"].apply(len) + df["response_a"].apply(len) + df["response_b"].apply(len)
df = df.sort_values(by=['len'])
df
```

</div>
<div class="column-right">

# 日本語訳

```python
import pandas as pd
df = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')  # データフレームにCSVファイルを読み込む
df = df.dropna()  # NaN値を削除
df = df.drop_duplicates(subset=['response_a', 'response_b'], keep=False)  # 重複する応答を削除
df["len"] = df["prompt"].apply(len) + df["response_a"].apply(len) + df["response_b"].apply(len)  # 各行の長さを計算
df = df.sort_values(by=['len'])  # 長さでソート
df
```

</div>
</details>

In [ ]:
import pandas as pd
df = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')  # データフレームにCSVファイルを読み込む
df = df.dropna()  # NaN値を削除
df = df.drop_duplicates(subset=['response_a', 'response_b'], keep=False)  # 重複する応答を削除
df["len"] = df["prompt"].apply(len) + df["response_a"].apply(len) + df["response_b"].apply(len)  # 各行の長さを計算
df = df.sort_values(by=['len'])  # 長さでソート
df

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
ds = Dataset.from_pandas(df)
ds = ds.select(torch.arange(1000)) #for demo purposes only
```

</div>
<div class="column-right">

# 日本語訳

```python
ds = Dataset.from_pandas(df)  # pandasデータフレームからデータセットを作成
ds = ds.select(torch.arange(1000))  # デモ目的のために最初の1000行を選択
```

</div>
</details>

In [ ]:
ds = Dataset.from_pandas(df)  # pandasデータフレームからデータセットを作成
ds = ds.select(torch.arange(1000))  # デモ目的のために最初の1000行を選択

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
class CustomTokenizer:
    def __init__(
        self, 
        tokenizer: PreTrainedTokenizerBase, 
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __call__(self, batch: dict) -> dict:
        prompt = ["Which is the better response for the prompt? response_a or response_b or tie? \n'n give score for each lable \n\n <prompt>: " + self.process_text(t) for t in batch["prompt"]]
        response_a = ["\n\n<response_a>: " + self.process_text(t) for t in batch["response_a"]]
        response_b = ["\n\n<response_b>: " + self.process_text(t) for t in batch["response_b"]]
        texts = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)
        labels=[]
        for a_win, b_win in zip(batch["winner_model_a"], batch["winner_model_b"]):
            if a_win:
                label = 0
            elif b_win:
                label = 1
            else:
                label = 2
            labels.append(label)
        return {**tokenized, "labels": labels}
        
    @staticmethod
    def process_text(text: str) -> str:
        return " ".join(eval(text, {"null": ""}))
```

</div>
<div class="column-right">

# 日本語訳

```python
class CustomTokenizer:
    def __init__(
        self, 
        tokenizer: PreTrainedTokenizerBase, 
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer  # トークナイザーを初期化
        self.max_length = max_length  # 最大長を設定
        
    def __call__(self, batch: dict) -> dict:
        # 各プロンプトのベターな応答を評価するためのテキストを作成
        prompt = ["Which is the better response for the prompt? response_a or response_b or tie? \n'n give score for each lable \n\n <prompt>: " + self.process_text(t) for t in batch["prompt"]]
        response_a = ["\n\n<response_a>: " + self.process_text(t) for t in batch["response_a"]]  # 応答Aを整形
        response_b = ["\n\n<response_b>: " + self.process_text(t) for t in batch["response_b"]]  # 応答Bを整形
        texts = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]  # プロンプト、応答A、応答Bを結合
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)  # テキストをトークン化
        labels=[]
        for a_win, b_win in zip(batch["winner_model_a"], batch["winner_model_b"]):  # 勝者を判別
            if a_win:
                label = 0  # 応答Aが勝った場合
            elif b_win:
                label = 1  # 応答Bが勝った場合
            else:
                label = 2  # 引き分けの場合
            labels.append(label)  # ラベルを追加
        return {**tokenized, "labels": labels}  # トークン化されたデータとラベルを返す
        
    @staticmethod
    def process_text(text: str) -> str:
        return " ".join(eval(text, {"null": ""}))  # テキストを処理して返す
```

</div>
</details>

In [ ]:
class CustomTokenizer:
    def __init__(
        self, 
        tokenizer: PreTrainedTokenizerBase, 
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer  # トークナイザーを初期化
        self.max_length = max_length  # 最大長を設定
        
    def __call__(self, batch: dict) -> dict:
        # 各プロンプトのベターな応答を評価するためのテキストを作成
        prompt = ["Which is the better response for the prompt? response_a or response_b or tie? \n'n give score for each lable \n\n <prompt>: " + self.process_text(t) for t in batch["prompt"]]
        response_a = ["\n\n<response_a>: " + self.process_text(t) for t in batch["response_a"]]  # 応答Aを整形
        response_b = ["\n\n<response_b>: " + self.process_text(t) for t in batch["response_b"]]  # 応答Bを整形
        texts = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]  # プロンプト、応答A、応答Bを結合
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)  # テキストをトークン化
        labels=[]
        for a_win, b_win in zip(batch["winner_model_a"], batch["winner_model_b"]):  # 勝者を判別
            if a_win:
                label = 0  # 応答Aが勝った場合
            elif b_win:
                label = 1  # 応答Bが勝った場合
            else:
                label = 2  # 引き分けの場合
            labels.append(label)  # ラベルを追加
        return {**tokenized, "labels": labels}  # トークン化されたデータとラベルを返す
        
    @staticmethod
    def process_text(text: str) -> str:
        return " ".join(eval(text, {"null": ""}))  # テキストを処理して返す

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
encode = CustomTokenizer(tokenizer, max_length=config.max_length)
ds = ds.map(encode, batched=True)
```

</div>
<div class="column-right">

# 日本語訳

```python
encode = CustomTokenizer(tokenizer, max_length=config.max_length)  # カスタムトークナイザーのインスタンスを作成
ds = ds.map(encode, batched=True)  # データセット全体に対してトークナイザーを適用
```

</div>
</details>

In [ ]:
encode = CustomTokenizer(tokenizer, max_length=config.max_length)  # カスタムトークナイザーのインスタンスを作成
ds = ds.map(encode, batched=True)  # データセット全体に対してトークナイザーを適用

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def compute_metrics(eval_preds: EvalPrediction) -> dict:
    preds = eval_preds.predictions
    labels = eval_preds.label_ids
    probs = torch.from_numpy(preds).float().softmax(-1).numpy()
    # Check for NaNs in predictions and labels
    if np.isnan(probs).any() or np.isnan(labels).any():
        raise ValueError("NaN values found in predictions or labels")

    loss = log_loss(y_true=labels, y_pred=probs)
    acc = accuracy_score(y_true=labels, y_pred=preds.argmax(-1))
    return {"acc": acc, "log_loss": loss}
```

</div>
<div class="column-right">

# 日本語訳

```python
def compute_metrics(eval_preds: EvalPrediction) -> dict:
    preds = eval_preds.predictions  # 予測値を取得
    labels = eval_preds.label_ids  # ラベルを取得
    probs = torch.from_numpy(preds).float().softmax(-1).numpy()  # 予測値から確率に変換
    # 予測値やラベルにNaNが含まれているかチェックする
    if np.isnan(probs).any() or np.isnan(labels).any():
        raise ValueError("NaN values found in predictions or labels")  # エラーを発生させる

    loss = log_loss(y_true=labels, y_pred=probs)  # ログ損失を計算
    acc = accuracy_score(y_true=labels, y_pred=preds.argmax(-1))  # 精度を計算
    return {"acc": acc, "log_loss": loss}  # 精度と損失を辞書型で返す
```

</div>
</details>

In [ ]:
def compute_metrics(eval_preds: EvalPrediction) -> dict:
    preds = eval_preds.predictions  # 予測値を取得
    labels = eval_preds.label_ids  # ラベルを取得
    probs = torch.from_numpy(preds).float().softmax(-1).numpy()  # 予測値から確率に変換
    # 予測値やラベルにNaNが含まれているかチェックする
    if np.isnan(probs).any() or np.isnan(labels).any():
        raise ValueError("NaN values found in predictions or labels")  # エラーを発生させる

    loss = log_loss(y_true=labels, y_pred=probs)  # ログ損失を計算
    acc = accuracy_score(y_true=labels, y_pred=preds.argmax(-1))  # 精度を計算
    return {"acc": acc, "log_loss": loss}  # 精度と損失を辞書型で返す

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
folds = [
        (
            [i for i in range(len(ds)) if i % config.n_splits != fold_idx],
            [i for i in range(len(ds)) if i % config.n_splits == fold_idx]
        ) 
        for fold_idx in range(config.n_splits)
    ]
```

</div>
<div class="column-right">

# 日本語訳

```python
folds = [
        (
            [i for i in range(len(ds)) if i % config.n_splits != fold_idx],  # トレーニングインデックス
            [i for i in range(len(ds)) if i % config.n_splits == fold_idx]  # 評価インデックス
        ) 
        for fold_idx in range(config.n_splits)  # 各分割について
    ]
```

</div>
</details>

In [ ]:
folds = [
        (
            [i for i in range(len(ds)) if i % config.n_splits != fold_idx],  # トレーニングインデックス
            [i for i in range(len(ds)) if i % config.n_splits == fold_idx]  # 評価インデックス
        ) 
        for fold_idx in range(config.n_splits)  # 各分割について
    ]

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from trl import SFTTrainer, SFTConfig
sft_config = SFTConfig(
    output_dir="output",
    overwrite_output_dir=True,
    report_to="none",
    num_train_epochs=config.n_epochs,
    per_device_train_batch_size=config.per_device_train_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    per_device_eval_batch_size=config.per_device_eval_batch_size,
    logging_steps=1000,
    save_strategy="epoch",
    save_steps=100,
    optim=config.optim_type,
    fp16=True,
    learning_rate=config.lr,
    warmup_steps=config.warmup_steps,
    packing=True, 
    dataset_text_field="text",
    max_seq_length=config.max_length,
)
```

</div>
<div class="column-right">

# 日本語訳

```python
from trl import SFTTrainer, SFTConfig
sft_config = SFTConfig(
    output_dir="output",  # 出力ディレクトリ
    overwrite_output_dir=True,  # 出力ディレクトリを上書きする設定
    report_to="none",  # レポートの設定
    num_train_epochs=config.n_epochs,  # トレーニングエポック数
    per_device_train_batch_size=config.per_device_train_batch_size,  # デバイスごとのトレーニングバッチサイズ
    gradient_accumulation_steps=config.gradient_accumulation_steps,  # 勾配累積ステップ数
    per_device_eval_batch_size=config.per_device_eval_batch_size,  # デバイスごとの評価バッチサイズ
    logging_steps=1000,  # ロギングのステップ数
    save_strategy="epoch",  # 保存戦略
    save_steps=100,  # 保存ステップ
    optim=config.optim_type,  # 最適化手法
    fp16=True,  # 半精度の使用
    learning_rate=config.lr,  # 学習率
    warmup_steps=config.warmup_steps,  # ウォームアップステップ
    packing=True,  # パッキングの設定
    dataset_text_field="text",  # データセット内のテキストフィールド名
    max_seq_length=config.max_length,  # 最大シーケンス長
)
```

</div>
</details>

In [ ]:
from trl import SFTTrainer, SFTConfig
sft_config = SFTConfig(
    output_dir="output",  # 出力ディレクトリ
    overwrite_output_dir=True,  # 出力ディレクトリを上書きする設定
    report_to="none",  # レポートの設定
    num_train_epochs=config.n_epochs,  # トレーニングエポック数
    per_device_train_batch_size=config.per_device_train_batch_size,  # デバイスごとのトレーニングバッチサイズ
    gradient_accumulation_steps=config.gradient_accumulation_steps,  # 勾配累積ステップ数
    per_device_eval_batch_size=config.per_device_eval_batch_size,  # デバイスごとの評価バッチサイズ
    logging_steps=1000,  # ロギングのステップ数
    save_strategy="epoch",  # 保存戦略
    save_steps=100,  # 保存ステップ
    optim=config.optim_type,  # 最適化手法
    fp16=True,  # 半精度の使用
    learning_rate=config.lr,  # 学習率
    warmup_steps=config.warmup_steps,  # ウォームアップステップ
    packing=True,  # パッキングの設定
    dataset_text_field="text",  # データセット内のテキストフィールド名
    max_seq_length=config.max_length,  # 最大シーケンス長
)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
trainer = SFTTrainer(
        model,
        train_dataset=ds,
        args=sft_config
    )
```

</div>
<div class="column-right">

# 日本語訳

```python
trainer = SFTTrainer(
        model,
        train_dataset=ds,  # トレーニングデータセットを設定
        args=sft_config  # 設定を渡す
    )
```

</div>
</details>

In [ ]:
trainer = SFTTrainer(
        model,
        train_dataset=ds,  # トレーニングデータセットを設定
        args=sft_config  # 設定を渡す
    )

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
for fold_idx in range(config.n_splits):
    
    train_idx, eval_idx = folds[fold_idx]

    train_data = ds.select(train_idx).sort("len")
    val_data = ds.select(eval_idx).sort("len")
    
    #split training data into batches with the same range of length
    batch_size = 200
    num_batches = len(train_data) // batch_size + (1 if len(train_data) % batch_size != 0 else 0)
    
    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = min(start_idx + batch_size, len(train_data))
        ds_temp = train_data.select(range(start_idx, end_idx))
        
        trainer.train_dataset = ds_temp
        
        print(f"Training batch {batch_idx + 1}/{num_batches} on fold {fold_idx + 1}/{config.n_splits}...")
        
        trainer.train()
        
        trainer.save_model(f"model_fold_{fold_idx}_batch{batch_idx}")

    
    # Validate after training on all batches
    trainer.eval_dataset = val_data
    
    print(f"Validating on fold {fold_idx + 1}/{config.n_splits}...")
    eval_results = trainer.evaluate()

    # Save metrics if needed
    print(f"Evaluation results for fold {fold_idx + 1}: {eval_results}")
```

</div>
<div class="column-right">

# 日本語訳

```python
for fold_idx in range(config.n_splits):  # 各分割に対してループ

    train_idx, eval_idx = folds[fold_idx]  # トレーニングインデックスと評価インデックスを取得

    train_data = ds.select(train_idx).sort("len")  # トレーニングデータを選択して長さでソート
    val_data = ds.select(eval_idx).sort("len")  # 評価データを選択して長さでソート
    
    # 同じ長さの範囲でトレーニングデータをバッチに分割
    batch_size = 200  # バッチサイズ
    num_batches = len(train_data) // batch_size + (1 if len(train_data) % batch_size != 0 else 0)  # バッチ数の計算
    
    for batch_idx in range(num_batches):  # 各バッチに対してループ
        start_idx = batch_idx * batch_size  # バッチの開始インデックス
        end_idx = min(start_idx + batch_size, len(train_data))  # バッチの終了インデックス
        ds_temp = train_data.select(range(start_idx, end_idx))  # 一時データセットを選択
        
        trainer.train_dataset = ds_temp  # トレーナーにトレーニングデータを設定
        
        print(f"Training batch {batch_idx + 1}/{num_batches} on fold {fold_idx + 1}/{config.n_splits}...")  # トレーニング進行状況を表示
        
        trainer.train()  # モデルのトレーニングを実行
        
        trainer.save_model(f"model_fold_{fold_idx}_batch{batch_idx}")  # モデルを保存

    
    # すべてのバッチでトレーニングした後は検証を実施
    trainer.eval_dataset = val_data  # 評価データを設定
    
    print(f"Validating on fold {fold_idx + 1}/{config.n_splits}...")  # 検証進行状況を表示
    eval_results = trainer.evaluate()  # 検証を実施

    # 必要に応じてメトリクスを保存
    print(f"Evaluation results for fold {fold_idx + 1}: {eval_results}")  # 評価結果を表示
```

</div>
</details>

In [ ]:
for fold_idx in range(config.n_splits):  # 各分割に対してループ

    train_idx, eval_idx = folds[fold_idx]  # トレーニングインデックスと評価インデックスを取得

    train_data = ds.select(train_idx).sort("len")  # トレーニングデータを選択して長さでソート
    val_data = ds.select(eval_idx).sort("len")  # 評価データを選択して長さでソート
    
    # 同じ長さの範囲でトレーニングデータをバッチに分割
    batch_size = 200  # バッチサイズ
    num_batches = len(train_data) // batch_size + (1 if len(train_data) % batch_size != 0 else 0)  # バッチ数の計算
    
    for batch_idx in range(num_batches):  # 各バッチに対してループ
        start_idx = batch_idx * batch_size  # バッチの開始インデックス
        end_idx = min(start_idx + batch_size, len(train_data))  # バッチの終了インデックス
        ds_temp = train_data.select(range(start_idx, end_idx))  # 一時データセットを選択
        
        trainer.train_dataset = ds_temp  # トレーナーにトレーニングデータを設定
        
        print(f"Training batch {batch_idx + 1}/{num_batches} on fold {fold_idx + 1}/{config.n_splits}...")  # トレーニング進行状況を表示
        
        trainer.train()  # モデルのトレーニングを実行
        
        trainer.save_model(f"model_fold_{fold_idx}_batch{batch_idx}")  # モデルを保存

    
    # すべてのバッチでトレーニングした後は検証を実施
    trainer.eval_dataset = val_data  # 評価データを設定
    
    print(f"Validating on fold {fold_idx + 1}/{config.n_splits}...")  # 検証進行状況を表示
    eval_results = trainer.evaluate()  # 検証を実施

    # 必要に応じてメトリクスを保存
    print(f"Evaluation results for fold {fold_idx + 1}: {eval_results}")  # 評価結果を表示